In [1]:
!pip install delta-spark==1.2.0

In [2]:
from pyspark.sql import SparkSession

In [3]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("DeltaApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 

spark = configure_spark_with_delta_pip(builder).master("spark://spark:7077").getOrCreate()

spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localstack:4566")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "foo")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "foo")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

In [4]:
data = spark.range(0, 5)
data.write.format("delta").save("s3a://mytestbucket/table")

In [5]:
df = spark.read.format("delta").load("s3a://mytestbucket/table")
df.show()

+---+
| id|
+---+
|  2|
|  3|
|  4|
|  0|
|  1|
+---+

